-----
# cartoGRAPHs - FIGURE 2A + Extended Data Figure 2
-----

In [1]:
#from cartoGRAPHs import *

%cd pyfiles 
from cartoGRAPHs import *

/Users/chris/Desktop/Github/cartoGRAPHs/pyfiles


2023-04-21 01:25:14.289173: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
%cd ..

/Users/chris/Desktop/Github/cartoGRAPHs


_____
# HUMAN PPI NETWORK
_____

In [3]:
organism = 'human'
G = load_graph(organism) 

d_gene_sym = load_genesymbols(G, organism)
l_features = list(d_gene_sym.values())

d_gene_do = pickle.load( open( "input/d_gene_do.pkl", "rb" ) )
d_do_genes = pickle.load( open( "input/d_do_genes.pkl", "rb" ) )
d_do_names = pickle.load( open( "input/DO_names.pkl", "rb" ) )
d_names_do = {y:x for x,y in d_do_names.items()}

d_centralities = load_centralities(G, organism)
df_centralities = pd.DataFrame(d_centralities).T
df_centralities.columns = ['degree','closeness', 'betweeness', 'eigenvector']
essential_genes,non_ess_genes,notdefined_genes = load_essentiality(G, organism)

In [4]:
G.name = "Human Interactome"
G.graph["desc"] = "The human interactome is a complex, dynamic network of protein-protein interactions that are essential for many biological processes. It is estimated that the human interactome consists of hundreds of thousands of interactions between thousands of proteins. These interactions can take many forms, including direct physical binding, allosteric regulation, and post-translational modifications. Many proteins participate in multiple interactions, forming highly interconnected clusters or modules within the network. The human interactome is not static and can change in response to cellular and environmental cues. "

_____ 
# FIGURE 2A | Importance layout (with diverse node lists highlighted)
_____

In [5]:
DM_cent = load_datamatrix(G,organism, 'importance')
DM_cent_round = round(DM_cent,4)

### PORTRAIT

In [6]:
%%time 

n_n = 10 #12 #20  
spr = 1.0 #1.0 #0.9  
md = 0.5 #0.1 #0.25 
Matrix = DM_cent_round 

posG_umap2D = layout_portrait_umap(G,
                                   Matrix,
                                   2,
                                   n_neighbors=n_n, 
                                   spread=spr, 
                                   min_dist=md, 
                                   metric='cosine'
                                  )
posG = posG_umap2D


posG_umap3D = layout_portrait_umap(G,
                                   Matrix,
                                   3,
                                   n_neighbors=n_n, 
                                   spread=spr, 
                                   min_dist=md, 
                                   metric='cosine'
                                  )
posG3D = posG_umap3D

CPU times: user 5min 22s, sys: 1min 12s, total: 6min 35s
Wall time: 5min 45s


---
## VISUAL SETTINGS + PLOT PREPARATION
---

#### NODES - GENERAL

In [7]:
opacity_nodes = 0.8
node_edge_col = '#696969' 
node_linewidth = 1

nodesglow_diameter = 8.0
nodesglow_transparency = 0.01 # 0.01

#### ESSENTIALITY color settings 

In [8]:
color_method = 'essentiality'

l_genes = essential_genes
    
node_col = '#00abff' 
undefined_col = '#DEDEDE '
rest_col_nodes = '#DEDEDE '

d_col_all = color_nodes_from_list(G, essential_genes, node_col)
colors = list(d_col_all.values())
    
edge_color = node_col 
d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)
width_edges = 0.25
opacity_edges = 0.25

print('Amount of Essential Genes', len(essential_genes))
print('Links inbetween Essential Genes', len(d_edge_col))

Amount of Essential Genes 2875
Links inbetween Essential Genes 9260


In [9]:
d_clustercolors = dict(zip(["Assoc. to Essential Gene","Not assoc. to Essential Gene"],set(d_col_all.values())))
d_clustercolors

{'Assoc. to Essential Gene': '#696969',
 'Not assoc. to Essential Gene': '#00abff'}

In [10]:
dict_for_cluster = {}
for g in G.nodes():
    if g in essential_genes:
        dict_for_cluster[g] = "Assoc. to Essential Gene"
    else:
        dict_for_cluster[g] = "Not assoc. to Essential Gene"

#### DEVELOPMENTAL color settings 

In [7]:
color_method = '8cell'

if color_method == 'oocyte' or color_method == '2cell' or color_method == '4cell' or color_method == '8cell':

    node_col = '#0181A3' 
    devgenes_selection = color_method
    df_devgenes = pd.read_csv('input/EmExplorer/'+devgenes_selection+'_Homo-sapiens.txt', delimiter = "\t")
    
    devgenes = list(df_devgenes['stage'])

    d_devgenes_entrezid = {}
    for i,k in d_gene_sym.items():
        if k in devgenes:
             d_devgenes_entrezid[k] = i 
    devgenes_entrezid = list(d_devgenes_entrezid.values())
    
    l_genes = devgenes_entrezid
    
    d_col_all = color_nodes_from_list(G, l_genes, node_col)
    colors = list(d_col_all.values())

    edge_color = node_col
    d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)
    width_edges = 0.25
    opacity_edges = 0.5
        
else:
    pass

In [8]:
print('Amount of Developmental Genes ('+ color_method + ')', len(devgenes_entrezid))
print('Links inbetween Developmental Genes ('+ color_method + ')', len(d_edge_col))

Amount of Developmental Genes (8cell) 38
Links inbetween Developmental Genes (8cell) 84


In [9]:
print('Amount of Developmental Genes ('+ color_method + ')', len(devgenes_entrezid))
print('Links inbetween Developmental Genes ('+ color_method + ')', len(d_edge_col))

Amount of Developmental Genes (8cell) 38
Links inbetween Developmental Genes (8cell) 84


In [10]:
print('Amount of Developmental Genes ('+ color_method + ')', len(devgenes_entrezid))
print('Links inbetween Developmental Genes ('+ color_method + ')', len(d_edge_col))

Amount of Developmental Genes (8cell) 38
Links inbetween Developmental Genes (8cell) 84


In [11]:
print('Amount of Developmental Genes ('+ color_method + ')', len(devgenes_entrezid))
print('Links inbetween Developmental Genes ('+ color_method + ')', len(d_edge_col))

Amount of Developmental Genes (8cell) 38
Links inbetween Developmental Genes (8cell) 84


#### CANCER color settings 

In [12]:
disease_category = 'cancer'

color_method = disease_category
disease_genes = get_disease_genes(G, d_names_do, d_do_genes, disease_category)
l_genes = disease_genes

node_col = '#00C9FF' 
d_col_all = color_nodes_from_list(G, disease_genes, node_col)
colors = list(d_col_all.values())

edge_color = node_col 
d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)

width_edges = 0.25
opacity_edges = 0.25

print('Amount of Cancer genes:' , len(disease_genes))
print('Links inbetween Cancer genes', len(d_edge_col))

Amount of Cancer genes: 2499
Links inbetween Cancer genes 26135


#### RARE DISEASE genes color settings 

In [13]:
import xml.etree.ElementTree as ET
tree = ET.parse('input/orphadata.xml')
root = tree.getroot()

# retrieve genes from all rare diseases from database/file  
disorder_to_genes = {}
for disorder in root.iter('Disorder'):
    dis = list(disorder.attrib.values())[0]
    for elem in disorder.findall('DisorderGeneAssociationList'):
        genes = []
        for disorderassoc in elem.findall('DisorderGeneAssociation'):
            for gene in disorderassoc.findall('Gene'):
                for reference in gene.findall('ExternalReferenceList'):
                     for refs in reference.findall('ExternalReference'):
                        for ensg in refs.findall('Source'):
                            if ensg.text == 'Ensembl':
                                for e in refs.findall('Reference'):
                                    ensgID = (e.text)
                            else:
                                pass
                            
            genes.append(ensgID)
    disorder_to_genes[dis] = genes

    
# get a list of all rare disease genes obtained 
all_raredisease_genes = []
for dis,genes in disorder_to_genes.items():
    for i in genes:
        all_raredisease_genes.append(i)
      
    
# make sure there are no duplicates
set_all_raredisease_genes = set(all_raredisease_genes)


# convert ensgID to entrezID and get gene symbols
ensg_to_entrez = pd.read_csv('input/ensg_to_entrezid.txt',sep="	")

ensg = list(ensg_to_entrez['From'])
entrez = [str(int(i)) for i in ensg_to_entrez['To'][:-1]] #list(ensg_to_entrez['To'])
d_ensg_entrez = dict(zip(ensg,entrez))


# match ensgID with entrezID
set_all_raredisease_genes_id = {}
for ix,entrez in d_ensg_entrez.items():
    if ix in set_all_raredisease_genes:
        set_all_raredisease_genes_id[ix]=entrez

        
# color all rare disease genes  
l_rarediseasegenes = list(set_all_raredisease_genes_id.values())
color_method = 'rarediseasegenes'

l_genes = l_rarediseasegenes

node_col =  '#004EAB'#'#0A81A1' 
undefined_col = '#DEDEDE '
rest_col_nodes = '#DEDEDE '

d_col_all = color_nodes_from_list(G, l_genes, node_col)
colors = list(d_col_all.values())
    
edge_color = node_col 
d_edge_col = color_edges_from_nodelist_specific(G, l_genes, node_col)
width_edges = 0.25
opacity_edges = 0.25

print('Amount of Rare disease genes:' , len(l_rarediseasegenes))
print('Links inbetween Rare disease genes:' , len(d_edge_col))

Amount of Rare disease genes: 4181
Links inbetween Rare disease genes: 33744


---
## PLOT NETWORK PORTRAIT
---

In [11]:
# plot nodes based on focus > separate to background/foreground 

posG_foreground = {}
posG_background = {}
for k,v in posG.items():
    if k in l_genes:
        posG_foreground[k]=v
    else: 
        posG_background[k]=v
    
d_colors_foreground = {}
d_colors_background = {}
for i,c in d_col_all.items():
    if i in posG_foreground.keys():
        d_colors_foreground[i]=c
    else: 
        d_colors_background[i]=c

colors_foreground = list(d_colors_foreground.values())
colors_background = list(d_colors_background.values())

d_feat_foreground = {}
d_feat_background = {}
for k,v in d_gene_sym.items():
    if k in posG_foreground: 
        d_feat_foreground[k]=v
    else:
        d_feat_background[k]=v
        
feat_foreground = list(d_feat_foreground.values())
feat_background = list(d_feat_background.values())

scale_factor = 0.025
d_size_plotly = draw_node_degree_3D(G, scale_factor)#/100)
d_size_plotly_foreground = {}
d_size_plotly_background = {}
for k,v in d_size_plotly.items():
    if k in posG_foreground.keys():
        d_size_plotly_foreground[k]=v
    else:
        d_size_plotly_background[k]=v
        
size_plotly_foreground = list(d_size_plotly_foreground.values())
size_plotly_background = list(d_size_plotly_background.values())

umap_nodes_foreground = get_trace_nodes_2D(posG_foreground, feat_foreground, colors_foreground, size_plotly_foreground, 
                                           node_linewidth)
umap_nodes_background = get_trace_nodes_2D(posG_background, feat_background, colors_background, size_plotly_background, 
                                           node_linewidth*0.25)
umap_nodes_background_ = get_trace_nodes_2D(posG_background, feat_background, colors_background, 0.5, node_linewidth*0.5)

umap_edges = get_trace_edges_specific2D(d_edge_col, posG, linew=width_edges, opac=opacity_edges)
data = [umap_edges, 
    umap_nodes_background_, umap_nodes_background, umap_nodes_foreground]

path = 'output/'
fname = '2Dportrait_NetlayoutImportance_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_'+color_method+'_'+organism

plot_2D(data,path,fname)

'output/2Dportrait_NetlayoutImportance_nn10_spr1.0_md0.5_essentiality_human.html'

In [12]:
umap_nodes = get_trace_nodes_3D(posG3D, list(d_gene_sym.values()), list(d_col_all.values()), list(d_size_plotly.values()), node_linewidth*0.5)
umap_edges = get_trace_edges_specific3D(d_edge_col, posG3D, linew=width_edges, opac=opacity_edges)
data = [umap_edges,umap_nodes]

fname3D = '3Dportrait_NetlayoutImportance_nn'+str(n_n)+'_spr'+str(spr)+'_md'+str(md)+'_'+color_method+'_'+organism

plot_3D(data,path,fname3D)

'output/3Dportrait_NetlayoutImportance_nn10_spr1.0_md0.5_essentiality_human.html'

---
## VRNETZER2 (multiplayer version) EXPORT
---

### EXPORT

In [13]:
# preparing input for functions

# link colors
d_linkcolors = {}
for edge in G.edges():
    if edge in d_edge_col.keys():
        d_linkcolors[edge]=d_edge_col[edge]
    else: 
        d_linkcolors[edge]=(0,0,0,0) 
        
# node annotations
l_annotations_csv = ['Gene: '+str(i)+'; ID:  '+str(j) for i,j in zip(d_gene_sym.values(), d_gene_sym.keys())]
l_annotations_json = [list(("Gene: "+str(i),"ID: "+str(j))) for i,j in zip(d_gene_sym.values(), d_gene_sym.keys())]
d_annotations_csv = dict(zip(list(G.nodes()),l_annotations_csv))
d_annotations_json = dict(zip(list(G.nodes()),l_annotations_json))

# clusterlabels 
dict_for_cluster_remap = dict(zip(range(0,len(G.nodes())), dict_for_cluster.values()))
clusterlabels = []
for cluster in d_clustercolors.keys():
    sublist = []
    for k,v in dict_for_cluster_remap.items():
        if cluster == v:
            sublist.append(str(k))
    sublist.insert(0,str(cluster))
    clusterlabels.append(sublist)

In [15]:
# ---------------
# 2D
#----------------
filename = 'VR/temp/'+'2A-'+G.name+'-essentiality2D'
exportVRNetzer_CSV(filename, G, posG, d_col_all, d_annotations_csv, d_linkcolors, clusterlabels)
exportVRNetzer_JSON(filename, G, posG, dict_for_cluster, d_col_all, d_annotations_json, d_linkcolors)

# ---------------
# 3D
#----------------  
filename = 'VR/temp/'+'2A-'+G.name+'-essentiality3D'
exportVRNetzer_CSV(filename, G, posG3D, d_col_all, d_annotations_csv, d_linkcolors, clusterlabels)
exportVRNetzer_JSON(filename, G, posG3D, dict_for_cluster, d_col_all, d_annotations_json, d_linkcolors)

Exported Files: 
 ['VR/temp/2A-HumanInteractome-essentiality2D_nodepositions.csv', 'VR/temp/2A-HumanInteractome-essentiality2D_nodecolors.csv', 'VR/temp/2A-HumanInteractome-essentiality2D_nodeproperties.csv', 'VR/temp/2A-HumanInteractome-essentiality2D_links.csv', 'VR/temp/2A-HumanInteractome-essentiality2D_linkcolors.csv', 'VR/temp/2A-HumanInteractome-essentiality2D_clusterlabels.csv']
Exported File: 
 ['VR/temp/2A-HumanInteractome-essentiality2D.json']
Exported Files: 
 ['VR/temp/2A-HumanInteractome-essentiality3D_nodepositions.csv', 'VR/temp/2A-HumanInteractome-essentiality3D_nodecolors.csv', 'VR/temp/2A-HumanInteractome-essentiality3D_nodeproperties.csv', 'VR/temp/2A-HumanInteractome-essentiality3D_links.csv', 'VR/temp/2A-HumanInteractome-essentiality3D_linkcolors.csv', 'VR/temp/2A-HumanInteractome-essentiality3D_clusterlabels.csv']
Exported File: 
 ['VR/temp/2A-HumanInteractome-essentiality3D.json']
